In [ ]:
import subprocess
import requests
import pyperclip

# Get staged changes
def get_git_diff():
    return subprocess.run(["git", "diff", "--cached"], capture_output=True, text=True).stdout

# Check if diff is large or contains binary files
def is_large_or_binary(diff):
    return any([
        "Binary files" in diff,
        any(ext in diff for ext in [".pdf", ".png", ".jpg", ".jpeg", ".zip", ".mp4", ".exe"]),
        len(diff) > 5000
    ])

# Build system prompt
def build_system_prompt():
    return """You are a commit message assistant. Your job is to write clear, structured Git commit messages based on a given diff.

Use this format:

<Title (max 70 chars)>

Added
  + Description

Changed
  * Description

Removed
  - Description

Only include sections that are present in the changes. Avoid excessive verbosity. Write messages that are readable in Git logs.

Example:

Add user analytics and cleanup legacy settings

Added
  + Event tracking for signup and login
  + Analytics integration with Mixpanel

Changed
  * Renamed `userFlow` to `userJourney`
  * Updated settings panel layout

Removed
  - Deprecated `oldTracking.js`
"""

# Talk to Ollama
def query_ollama(diff, model="gemma3:latest"):
    prompt = f"Write a structured Git commit message for the following diff:\n\n{diff}"
    response = requests.post("http://localhost:11434/api/generate", json={
        "model": model,
        "prompt": prompt,
        "system": build_system_prompt(),
        "stream": False
    })
    return response.json()["response"].strip()

# Main logic
if __name__ == "__main__":
    diff = get_git_diff()

    if not diff.strip():
        print("⚠️ No staged changes.")
    elif is_large_or_binary(diff):
        print("⚠️ Diff too large or contains binary files. Skipping AI commit message.")
    else:
        message = query_ollama(diff)
        print("\n💡 Suggested Commit Message:\n", message)

        pyperclip.copy(message)
        print("📋 Copied to clipboard!")